In [6]:
import os
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()

True

# Time Series Forecasting
## AutoMLV2

- https://sagemaker-core.readthedocs.io/en/stable/
- https://aws.amazon.com/blogs/machine-learning/time-series-forecasting-with-amazon-sagemaker-automl/
- https://github.com/aws-samples/time-series-forecasting-with-automlv2/blob/main/automlv2_forecasting.ipynb

In [2]:
# import os
# import boto3

# os.environ["AWS_PROFILE"] = "aptimyz-env01"
# os.environ["AWS_DEFAULT_REGION"] = "eu-central-1"

# boto3.setup_default_session()

In [ ]:
# from datetime import datetime
# import uuid
# import time

# from sagemaker_core.helper.session_helper import Session, get_execution_role


# sagemaker_session = Session()

# region = sagemaker_session.boto_region_name
# role = get_execution_role(sagemaker_session=sagemaker_session)

# print(f"AWS region: {region}")
# print(f"Execution role: {role}")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/khozzy/Library/Application Support/sagemaker/config.yaml


## Training Pipeline

In [ ]:
INPUT_S3_BUCKET = os.environ["SM_INPUT_BUCKET"]
OUTPUT_S3_BUCKET = os.environ["SM_OUTPUT_BUCKET"]

In [ ]:
from sagemaker.automl.automlv2 import (
    AutoMLV2,
    AutoMLTimeSeriesForecastingConfig,
    AutoMLDataChannel,
)

now = datetime.now().strftime("%Y%m%d-%H%M")
job_name = f"train-{now}"

time_series_config = AutoMLTimeSeriesForecastingConfig(
    forecast_frequency="D",
    forecast_horizon=7,
    forecast_quantiles=["p10", "p50", "p90"],
    target_attribute_name="sales",
    timestamp_attribute_name="date",
    item_identifier_attribute_name="item",
    grouping_attribute_names=["store"],
    max_candidates=1,
    max_runtime_per_training_job_in_seconds=600,
    max_total_job_runtime_in_seconds=3600,
)

# job objectives
# https://docs.aws.amazon.com/sagemaker/latest/dg/autopilot-metrics-validation.html
automl_sm_job = AutoMLV2(
    problem_config=time_series_config,
    base_job_name=job_name,
    role=os.environ["SM_TRAIN_ROLE_ARN"],
    output_path=f"s3://{OUTPUT_S3_BUCKET}/",
)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/khozzy/Library/Application Support/sagemaker/config.yaml


In [9]:
# takes about 30 minutes
automl_sm_job.fit(
    inputs=[
        AutoMLDataChannel(
            s3_data_type="S3Prefix",
            s3_uri=f"s3://{INPUT_S3_BUCKET}/data/train.csv",
            channel_type="training",
            content_type="text/csv;header=present",
        )
    ],
    wait=True,
    logs=True,
)

{"AutoMLJobName": "train-20-2025-09-23-11-04-01-356", "AutoMLJobInputDataConfig": [{"DataSource": {"S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": "s3://sagemaker-forecasting-aptimyz-input/data/train.csv"}}, "ChannelType": "training", "ContentType": "text/csv;header=present"}], "OutputDataConfig": {"S3OutputPath": "s3://sagemaker-forecasting-aptimyz-output/"}, "AutoMLProblemTypeConfig": {"TimeSeriesForecastingJobConfig": {"CompletionCriteria": {"MaxCandidates": 1, "MaxRuntimePerTrainingJobInSeconds": 600, "MaxAutoMLJobRuntimeInSeconds": 3600}, "ForecastHorizon": 7, "ForecastFrequency": "D", "TimeSeriesConfig": {"TargetAttributeName": "sales", "TimestampAttributeName": "date", "ItemIdentifierAttributeName": "item", "GroupingAttributeNames": ["store"]}, "ForecastQuantiles": ["p10", "p50", "p90"]}}, "RoleArn": "arn:aws:iam::283674368847:role/SageMakerTrainingExecutionRole", "SecurityConfig": {"EnableInterContainerTrafficEncryption": false}}
............................................

## Retrieve best candidate

In [1]:
best_candidate = automl_sm_job.best_candidate()
best_candidate_name = best_candidate["CandidateName"]
print("BestCandidateName:", best_candidate_name)

NameError: name 'automl_sm_job' is not defined

In [11]:
# creates a model wrapper
automl_model = automl_sm_job.create_model(
    name=best_candidate_name, candidate=best_candidate
)

automl_model.create(instance_type="ml.m5.xlarge")

## Batch predictions (transformer inference)

Let's evaluate the performance of the best model by predicting next values using after `train` data and later compare it with the ground truth.

In [16]:
from sagemaker.transformer import Transformer

transformer = Transformer(
    model_name=best_candidate_name,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    output_path=f"s3://{OUTPUT_S3_BUCKET}/transform/{best_candidate_name}/",
)

transformer.transform(
    data=f"s3://{INPUT_S3_BUCKET}/data/train.csv",
    content_type="text/csv",
    split_type="Line",
)

INFO:sagemaker:Creating transform job with name: autopilot-timeseries-2025-09-23-12-56-07-401


....................................[2025-09-23 13:02:16] [INFO] matplotlib.font_manager generated new fontManager
[2025-09-23 13:02:16] [ERROR] prophet.plot Importing plotly failed. Interactive plots will not work.
[2025-09-23 13:02:16] [INFO] rpy2.situation cffi mode is CFFI_MODE.ANY
[2025-09-23 13:02:16] [INFO] rpy2.situation R home found: /usr/lib64/R
[2025-09-23 13:02:16] [INFO] rpy2.situation R library path: /usr/lib64/R/lib:/usr/lib/jvm/jre/lib/amd64/server:/usr/lib/jvm/jre/lib/amd64:/usr/lib/jvm/java/lib/amd64:/usr/lib/jvm/jre/lib/server:/usr/java/packages/lib/amd64:/lib:/usr/lib:/opt/amazon/lib
[2025-09-23 13:02:16] [INFO] rpy2.situation LD_LIBRARY_PATH: /opt/amazon/lib
[2025-09-23 13:02:16] [INFO] rpy2.rinterface_lib.embedded Default options to initialize R: rpy2, --quiet, --no-save
[2025-09-23 13:02:17] [INFO] rpy2.rinterface_lib.embedded R is already initialized. No need to initialize.
[09/23/2025 13:02:18 INFO 139886154118976] Run 'serve' command
[2025-09-23 13:02:18] [INF